# Cuaderno 01: Generación de Datasets Auditivos y Visuales

**Objetivo:** Crear los datasets paralelos que servirán de base para todo el proyecto:
1.  **Dataset Auditivo**: Generar archivos de audio `.wav` para una lista de fonemas/letras clave.
2.  **Dataset Visual**: Descargar y extraer imágenes de letras (`.png`) del dataset **EMNIST** que correspondan a los grafemas de nuestras listas.

**Flujo de Trabajo:**
1.  **Instalación de Librerías**: Instalar `gTTS`, `pydub` y `torchvision`.
2.  **Configuración**: Definir las listas de fonemas/grafemas y las rutas de salida para ambos datasets.
3.  **Parte A**: Generar los archivos de audio.
4.  **Parte B**: Descargar, filtrar y guardar las imágenes de letras correspondientes.

## Paso 1: Instalación de Librerías

Instalamos las librerías necesarias:
* `gTTS` y `pydub`: Para la síntesis de voz y manejo de audio.
* `torch` y `torchvision`: Para descargar y manipular el dataset de imágenes EMNIST.

In [ ]:
%pip install gTTS pydub torch torchvision matplotlib

## Paso 2: Configuración General

Importamos librerías y definimos las listas de fonemas/grafemas y las rutas de salida. Ahora incluimos rutas para los datos visuales.

In [2]:
import os
from pathlib import Path
from gtts import gTTS
from pydub import AudioSegment
import torch
import torchvision
from torchvision.datasets import EMNIST
from PIL import Image

# --- Rutas de Salida ---
project_root = Path.cwd().parent

# Rutas para el dataset auditivo
output_dir_audio_es = project_root / "data/02_processed/phoneme_audio/es"
output_dir_audio_en = project_root / "data/02_processed/phoneme_audio/en"
output_dir_audio_es.mkdir(parents=True, exist_ok=True)
output_dir_audio_en.mkdir(parents=True, exist_ok=True)

# Rutas para el dataset visual
output_dir_emnist_root = project_root / "data/01_raw/emnist"
output_dir_visual_es = project_root / "data/02_processed/grapheme_images/es"
output_dir_visual_en = project_root / "data/02_processed/grapheme_images/en"
output_dir_visual_es.mkdir(parents=True, exist_ok=True)
output_dir_visual_en.mkdir(parents=True, exist_ok=True)

# --- Listas de Fonemas/Grafemas ---
phonemes_es = [
    'a', 'e', 'i', 'o', 'u', 'b', "c", 'd', 'f', 'g', 'j',"h", 'k', 'l', 'm', 'n', 'ñ', 'p', 'r', 'rr', 's', 't', 'y',"v", 'z', 'ch', 'll'
]
phonemes_en = [
    'a', 'e', 'i', 'o', 'u', 'ay', 'ee', 'igh', 'oh', 'oo', 'b',"c", 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'r', 's', 't', 'v', 'w', 'y', 'z', 'ch', 'sh', 'th', 'ng'
]

# --- Parámetros de Procesamiento ---
MAX_IMAGES_PER_LETTER = 50

print(f"Se guardará un máximo de {MAX_IMAGES_PER_LETTER} imágenes por grafema.")
print(f"Fonemas a procesar para español: {len(phonemes_es)}")
print(f"Fonemas a procesar para inglés: {len(phonemes_en)}")

Se guardará un máximo de 50 imágenes por grafema.
Fonemas a procesar para español: 27
Fonemas a procesar para inglés: 33


## Parte A: Generación del Dataset Auditivo

Esta sección genera los archivos `.wav` para cada fonema/letra de nuestras listas.

In [3]:
def generate_audio_files(phoneme_list, lang_code, output_path):
    print(f"\n--- Iniciando generación de audio para: {lang_code.upper()} ---")
    for i, phoneme in enumerate(phoneme_list):
        wav_filepath = output_path / f"{phoneme}.wav"
        if wav_filepath.exists():
            print(f"({i+1}/{len(phoneme_list)}) Audio para '{phoneme}' ya existe. Omitiendo.")
            continue
        try:
            tts = gTTS(text=phoneme, lang=lang_code, slow=True)
            mp3_temp_path = output_path / "temp.mp3"
            tts.save(mp3_temp_path)
            audio = AudioSegment.from_mp3(mp3_temp_path)
            audio.export(wav_filepath, format="wav", parameters=["-ac", "1", "-ar", "16000"])
            os.remove(mp3_temp_path)
            print(f"({i+1}/{len(phoneme_list)}) Audio para '{phoneme}' guardado.")
        except Exception as e:
            print(f"Error procesando '{phoneme}': {e}")
            if os.path.exists(mp3_temp_path): os.remove(mp3_temp_path)

# Generar audios para ambos idiomas
generate_audio_files(phonemes_es, 'es', output_dir_audio_es)
generate_audio_files(phonemes_en, 'en', output_dir_audio_en)
print("\n--- Proceso de generación de audio completado. ---")


--- Iniciando generación de audio para: ES ---
(1/27) Audio para 'a' guardado.
(2/27) Audio para 'e' guardado.
(3/27) Audio para 'i' guardado.
(4/27) Audio para 'o' guardado.
(5/27) Audio para 'u' guardado.
(6/27) Audio para 'b' guardado.
(7/27) Audio para 'c' guardado.
(8/27) Audio para 'd' guardado.
(9/27) Audio para 'f' guardado.
(10/27) Audio para 'g' guardado.
(11/27) Audio para 'j' guardado.
(12/27) Audio para 'h' guardado.
(13/27) Audio para 'k' guardado.
(14/27) Audio para 'l' guardado.
(15/27) Audio para 'm' guardado.
(16/27) Audio para 'n' guardado.
(17/27) Audio para 'ñ' guardado.
(18/27) Audio para 'p' guardado.
(19/27) Audio para 'r' guardado.
(20/27) Audio para 'rr' guardado.
(21/27) Audio para 's' guardado.
(22/27) Audio para 't' guardado.
(23/27) Audio para 'y' guardado.
(24/27) Audio para 'v' guardado.
(25/27) Audio para 'z' guardado.
(26/27) Audio para 'ch' guardado.
(27/27) Audio para 'll' guardado.

--- Iniciando generación de audio para: EN ---
(1/33) Audio para '

## Parte B: Generación del Dataset Visual

Ahora, descargamos el dataset EMNIST y extraemos las imágenes de las letras que corresponden a nuestros grafemas de una sola letra (ej. 'a', 'b', 'c'). Las imágenes se guardarán en subcarpetas por cada letra.

In [4]:
def generate_visual_dataset(phoneme_list, emnist_dataset, output_path, max_images):
    """
    Filtra EMNIST y guarda un número máximo de imágenes por grafema.
    """
    class_to_char = {i: c for i, c in enumerate(emnist_dataset.classes) if c.isalpha()}
    char_to_class = {c: i for i, c in class_to_char.items()}
    
    single_char_graphemes = [p for p in phoneme_list if len(p) == 1 and p.isalpha()]
    
    print(f"\n--- Extrayendo un máximo de {max_images} imágenes para {len(single_char_graphemes)} grafemas ---")
    
    images_by_label = {label: [] for label in range(len(emnist_dataset.classes))}
    for image, label in emnist_dataset:
        images_by_label[label].append(image)

    for i, grapheme in enumerate(single_char_graphemes):
        grapheme_lower = grapheme.lower()
        if grapheme_lower in char_to_class:
            class_idx = char_to_class[grapheme_lower]
            grapheme_dir = output_path / grapheme_lower
            grapheme_dir.mkdir(parents=True, exist_ok=True)
            
            num_existing = len(list(grapheme_dir.glob("*.png")))
            if num_existing >= max_images:
                print(f"({i+1}/{len(single_char_graphemes)}) Límite de imágenes para '{grapheme_lower}' ya alcanzado. Omitiendo.")
                continue

            images_saved_count = 0
            for img_obj in images_by_label[class_idx]:
                if images_saved_count >= max_images:
                    break
                
                img = img_obj.transpose(Image.Transpose.ROTATE_270)
                img = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
                # Usamos el contador actual para el nombre del archivo
                img.save(grapheme_dir / f"{grapheme_lower}_{num_existing + images_saved_count}.png")
                images_saved_count += 1
            
            print(f"({i+1}/{len(single_char_graphemes)}) Se guardaron {images_saved_count} imágenes para '{grapheme_lower}'.")
        else:
            print(f"Grafema '{grapheme_lower}' no encontrado en EMNIST 'letters'.")

# --- Descargar el dataset EMNIST una sola vez ---
print("\n--- Descargando y preparando EMNIST ---")
emnist_data = EMNIST(root=output_dir_emnist_root, split='letters', download=True)
            
# --- CORRECCIÓN: Llamadas a la función actualizadas ---
# Añadimos el argumento 'max_images' que ahora es requerido
generate_visual_dataset(phonemes_es, emnist_data, output_dir_visual_es, max_images=MAX_IMAGES_PER_LETTER)
generate_visual_dataset(phonemes_en, emnist_data, output_dir_visual_en, max_images=MAX_IMAGES_PER_LETTER)

print("\n--- Proceso de generación de imágenes completado. ---")


--- Descargando y preparando EMNIST ---


100%|██████████| 562M/562M [28:48<00:00, 325kB/s]   



--- Extrayendo un máximo de 50 imágenes para 24 grafemas ---
(1/24) Límite de imágenes para 'a' ya alcanzado. Omitiendo.
(2/24) Límite de imágenes para 'e' ya alcanzado. Omitiendo.
(3/24) Límite de imágenes para 'i' ya alcanzado. Omitiendo.
(4/24) Límite de imágenes para 'o' ya alcanzado. Omitiendo.
(5/24) Límite de imágenes para 'u' ya alcanzado. Omitiendo.
(6/24) Límite de imágenes para 'b' ya alcanzado. Omitiendo.
(7/24) Límite de imágenes para 'c' ya alcanzado. Omitiendo.
(8/24) Límite de imágenes para 'd' ya alcanzado. Omitiendo.
(9/24) Límite de imágenes para 'f' ya alcanzado. Omitiendo.
(10/24) Límite de imágenes para 'g' ya alcanzado. Omitiendo.
(11/24) Límite de imágenes para 'j' ya alcanzado. Omitiendo.
(12/24) Límite de imágenes para 'h' ya alcanzado. Omitiendo.
(13/24) Límite de imágenes para 'k' ya alcanzado. Omitiendo.
(14/24) Límite de imágenes para 'l' ya alcanzado. Omitiendo.
(15/24) Límite de imágenes para 'm' ya alcanzado. Omitiendo.
(16/24) Límite de imágenes para 